In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2017.csv")

d:\piggy\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (18,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10803028 entries, 0 to 10803027
Data columns (total 43 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   Summons Number                     int64  
 1   Plate ID                           object 
 2   Registration State                 object 
 3   Plate Type                         object 
 4   Issue Date                         object 
 5   Violation Code                     int64  
 6   Vehicle Body Type                  object 
 7   Vehicle Make                       object 
 8   Issuing Agency                     object 
 9   Street Code1                       int64  
 10  Street Code2                       int64  
 11  Street Code3                       int64  
 12  Vehicle Expiration Date            int64  
 13  Violation Location                 float64
 14  Violation Precinct                 int64  
 15  Issuer Precinct                    int64  
 16  Issuer Code     

# TASK #1

## Dimension 1 (Consistency)

1. The numbers of houses exist for every row.  

2. "Registration state" exist for every row as two letters (example “CA”). 

In [4]:
df['House Number'].isnull().sum()

2288618

In [5]:
(df['Registration State'].apply(len) != 2).sum()

0

In [6]:
df['Registration State'].sort_values().unique()

array(['99', 'AB', 'AK', 'AL', 'AR', 'AZ', 'BC', 'CA', 'CO', 'CT', 'DC',
       'DE', 'DP', 'FL', 'FO', 'GA', 'GV', 'HI', 'IA', 'ID', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MB', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS',
       'MT', 'MX', 'NB', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NS', 'NT',
       'NV', 'NY', 'OH', 'OK', 'ON', 'OR', 'PA', 'PE', 'PR', 'QB', 'RI',
       'SC', 'SD', 'SK', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV',
       'WY'], dtype=object)

In [7]:
df[df['Registration State'] == '99']['Registration State'].count()

36625

## Dimension 2 (Timeliness)
1. We do not need data from any year but 2017.


In [8]:
pd.to_datetime(df['Issue Date']).dt.year.value_counts().head(7)

2017    5431918
2016    5368391
2018       1057
2019        472
2015        419
2000        185
2014        120
Name: Issue Date, dtype: int64

## Dimension 3 (Completeness)
1. House where violation was captured is present;
2. "Violation description" should exist.


In [9]:
df[['House Number', 'Violation Description']].isnull().sum()

House Number             2288618
Violation Description    1127470
dtype: int64

## Dimension 4 (Uniqueness)
1. No double rows, no duplicates.

In [10]:
df.duplicated().sum()

0

## Dimension 5 (Validity)
1. Data types are as described by business:
- Numbers: Summons Number, Violation Code, Street Code1, Street Code2, Street Code3, Vehicle Expiration Date, Violation Precinct,  
Issuer Precinct, Issuer Code, Date First Observed, Law Section, Vehicle Year, Feet From Curb;
- Date & Time: Issue Date;
- Plain text: all the rest columns.

Additionally: Check for outliers (deviations);


In [11]:
num_clmn = ['Summons Number', 'Violation Code', 'Street Code1', 'Street Code2', 'Street Code3', 'Vehicle Expiration Date', 'Violation Precinct',
'Issuer Precinct', 'Issuer Code', 'Date First Observed', 'Law Section', 'Vehicle Year', 'Feet From Curb']
dt_clmn = ['Issue Date']
text_clmn = list(set(df.columns) - set(num_clmn) - set(dt_clmn))

numbers = []
date_time = []
p_text = []
for c in df.columns:
    if df[c].dtype == 'int64' or df[c].dtype == 'float64':
        numbers.append(c)
    elif df[c].dtype == 'datetime64':
        date_time.append(c)
    else:
        p_text.append(c)

In [12]:
wrong_type = {}
def check_type(set1, set2, types):
    for s in set1:
        if s not in set2:
            wrong_type[s] = types
            
check_type(num_clmn, numbers, 'int64')  
check_type(dt_clmn, date_time, 'datetime64')
check_type(text_clmn, p_text, 'object')

In [13]:
wrong_type

{'Issue Date': 'datetime64',
 'Unregistered Vehicle?': 'object',
 'No Standing or Stopping Violation': 'object',
 'Violation Location': 'object',
 'Hydrant Violation': 'object',
 'Double Parking Violation': 'object'}

**Additionally: Check for outliers (deviations);**

In [14]:
for c in df.columns:
    print(c) if (pd.api.types.infer_dtype(df[c]) == 'mixed') else None

Issuer Squad
Violation Post Code


## Dimension 6 (Accuracy)
1. The numbers of houses are real.

In [15]:
df['House Number'].head(20)

0      NaN
1      NaN
2      NaN
3      330
4      799
5      NaN
6      439
7      NaN
8      NaN
9      NaN
10     126
11    1365
12     NaN
13    1224
14     279
15     143
16     NaN
17     204
18     330
19     NaN
Name: House Number, dtype: object

In [16]:
df[df['House Number'].str.lower().isin(['one','two', 'three','four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'fifteen',
                                        'hundred', 'thousend', 'zero'])][['House Number']].count()

House Number    237
dtype: int64

In [17]:
df[df['House Number'].str.contains('west|east|south|north| st| ave| sq| blvd| dr| pl| rd| cres',na=False, case=False, regex=True)][['House Number']].count()

House Number    55
dtype: int64

In [18]:
df[~df['House Number'].str.contains('1|2|3|4|5|6|7|8|9|0',na=False, case=False, regex=True)][['House Number']].notnull().count()

House Number    3472207
dtype: int64

## Correction DF

### c1.1
После первой проверки удаляем все строки, если отсутствует значение 'House Number' и 'Registration State'.
А название штата не состоит из двух букв.

In [19]:
df_corr = df.dropna(subset=['House Number', 'Registration State'])

In [20]:
# df_corr = df_corr[df_corr['Registration State'] != '99']

#### c1.1.1
Проверка название штата состоит из двух букв с помощью regex.

In [21]:
df_corr = df_corr[df_corr['Registration State'].str.contains(r'\D{2}', regex=True)]

In [56]:
df_corr['Registration State'].sort_values().unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

### c1.2
Второй проверкой удаляем данные за все годы кроме 2017

In [23]:
df_corr = df_corr.loc[pd.to_datetime(df_corr['Issue Date']).dt.year == 2017]

### c1.3
Третья проверка наличие описания нарушения при условии заполненности 'House Number'

In [24]:
df_corr = df_corr.dropna(subset=['House Number', 'Violation Description'])

### c1.4
Удаление явных дубликатов строк

In [25]:
df_corr.drop_duplicates(inplace=True)

### c1.5
Приведение типов данных согласно бизнес требованиям

In [26]:
df_corr = df_corr.astype(wrong_type)

### c1.6
Проверка соответствия номеров домов "реальным" значениям.  
Удалены из номеров домов символы [.-/ + `], начальные нули.  
Номера представленные в текстовом виде('one','two', 'three' etc) также удалены, но также можно с помощью функции их конвертировать в число, если бы их количество составляло значительную долю от всего DF.
Удалены номера в которых отсутвуют числа (номера представлены только буквами).  
Удалены строки содержащие кроме номера дома дополнительные данные (улица, район и т.д.)

In [27]:
df_corr['House Number'] = (df_corr['House Number']
            .replace(to_replace=r"[/'+._`]", value='', regex=True)
            .replace(to_replace="^0{1,5}", value='', regex=True)
            .replace(to_replace="^-{1,2}", value='', regex=True)
            .replace(to_replace="-{2,}", value='-', regex=True)
            .replace(to_replace="-$", value='', regex=True)
            .replace(to_replace="^0{1,5}", value='', regex=True)
           )


In [28]:
df_corr = df_corr[~df_corr['House Number'].str.contains(
    'west|east|south|north| st| ave| sq| blvd| dr| pl| rd| cres', na=False, case=False, regex=True)]

In [29]:
df_corr = df_corr[df_corr['House Number'].str.contains('1|2|3|4|5|6|7|8|9|0',na=False, case=False, regex=True)]

In [30]:
df_corr = df_corr[~df_corr['House Number'].str.lower().isin(['one','two', 'three','four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'fifteen',
                                        'hundred', 'thousend', 'zero'])]

# TASK #2

Using data described above (2017 year only) filter it according to business rules. As a result, you should receive check results for every business rule with DataFrame containing deviations.  

Business rules that out data MUST follow:
1. 'Vehicle Make' column should not be 'TOYOT'.
2. No fully nulls columns.
3. No nulls or 'BLANKPLATE' in the Plate ID column.
4. "Registration state": should be only from 50 USA states, named as in ANSI standard INCITS 38:2009 (https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations).
5. "Plate type" only PAS, COM.
6. Issue date - convert to dates. Find min/max.
7. No unregistered vehicle in the "Unregistered Vehicle" column (1 means unregistered vehicle, 0 means registered vehicle).


### c2.1
Удалены строки с производителем TOYOT

In [31]:
df_corr['Vehicle Make'].value_counts()

FORD     420363
TOYOT    341399
HONDA    312563
NISSA    258975
CHEVR    231048
          ...  
ACCO          1
CAMR          1
ULTR          1
FRH           1
BAYB          1
Name: Vehicle Make, Length: 235, dtype: int64

In [32]:
df_corr = df_corr[df_corr['Vehicle Make']!= 'TOYOT']

### c2.2
Удалены строки полностью заполненные nan значениями

In [33]:
df.dropna(how='all')

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,...,BK,NaN,2004,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
3,8478629828,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,...,WH,NaN,2007,NaN,0,04,47-Double PKG-Midtown,NaN,NaN,NaN
4,7868300310,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,...,WHITE,NaN,2007,NaN,0,31 6,69-Failure to Disp Muni Recpt,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10803023,1415891400,HGK6453,NJ,PAS,11/02/2068,21,SDN,HONDA,S,17380,...,BLACK,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
10803024,1384716543,GRA6240,NY,PAS,07/12/2069,20,SUBN,TOYOT,X,93830,...,GRAY,0.0,2007,-,0,NaN,NaN,NaN,NaN,NaN
10803025,1413536554,RC8S28,PA,PAS,08/14/2069,46,SUBN,NaN,P,37980,...,RED,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
10803026,1415514203,HGU9544,NY,PAS,11/15/2069,40,SUBN,JEEP,P,0,...,BROWN,0.0,2011,-,0,NaN,NaN,NaN,NaN,NaN


#### c2.2.1
Удалены колонки полностью заполненные nan значениями

In [34]:
len(df_corr.columns)

43

In [35]:
df_corr.dropna(axis=1, how='all', inplace=True)

In [36]:
len(df_corr.columns)

39

### c2.3
Проверка заполненности столбца Plate ID

In [37]:
df_corr[df_corr['Plate ID'].str.lower() == 'BLANKPLATE'.lower()]['Plate ID'].count()

0

In [38]:
df_corr[df_corr['Plate ID'].isnull()]['Plate ID'].count()

0

In [39]:
df_corr = df_corr[df_corr['Plate ID'].str.lower() != 'BLANKPLATE'.lower()][~df_corr['Plate ID'].isnull()]

### c2.4
Проверка правильности заполнения столбца 'Registration State' в соответствии с ANSI standard INCITS 38:2009

In [40]:
us_states = pd.read_csv('codes.csv', sep=';')
us_states.columns = ['name', 'num', 'code']
us_states.head()

,name,num,code
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


In [41]:
wrong_states = df[~df['Registration State'].isin(us_states['code'])]['Registration State'].value_counts()
wrong_states.size, wrong_states

(16,
 99    36625
 ON     5601
 QB     4406
 DP     3192
 NS      730
 GV      704
 AB      218
 NB      137
 PE      128
 BC      127
 PR      110
 MB       40
 SK       30
 FO       12
 MX        4
 NT        4
 Name: Registration State, dtype: int64)

In [42]:
df_corr = df_corr[df_corr['Registration State'].isin(us_states['code'])]

### c2.5
Проверка правильности заполнения столбца 'Plate type', допустимые значения PAS и COM

In [43]:
df['Plate Type'].value_counts()

PAS    7993499
COM    1839534
OMT     450185
OMS     108568
SRF      88713
        ...   
GAC          2
JWV          1
HOU          1
JCA          1
LOC          1
Name: Plate Type, Length: 86, dtype: int64

In [44]:
df_corr  = df_corr[df_corr['Plate Type'].isin(['PAS','COM'])]

### c2.6
Приведение к дате столбца Issue date, нахождение максимального и минимального значения

In [45]:
df_corr['Issue Date'].dtype

dtype('<M8[ns]')

In [46]:
df_corr['Issue Date'].max(), df_corr['Issue Date'].min()

(Timestamp('2017-06-27 00:00:00'), Timestamp('2017-01-01 00:00:00'))

In [47]:
df_corr['Issue Date'] = df_corr['Issue Date'].dt.date

In [48]:
print(f"максимальная дата DF: {df_corr['Issue Date'].max()}\nминимальная дата DF: {df_corr['Issue Date'].min()}")

максимальная дата DF: 2017-06-27
минимальная дата DF: 2017-01-01


In [49]:
df_corr['Issue Date'].value_counts()

2017-06-15    24312
2017-06-20    23488
2017-03-21    23250
2017-06-22    23093
2017-06-02    23017
              ...  
2017-01-02     1917
2017-06-25     1543
2017-01-01     1151
2017-01-08      604
2017-02-09      540
Name: Issue Date, Length: 177, dtype: int64

### c2.7
Проверка отсутствия в DF незарегистрированных транспортных средств.  
90% первоначального DF имеет пропуски данных в столбце 'Unregistered Vehicle?', обработанный DF имеет заполнение только в 120 строках (менее 1%).  
Варианты дальнейшей обработки:  
- заполнить пропуски также 0, т.к. можно предположить отсутствие явного указания значения признака == True (1) трактовать как == False (0)  
- обрабатывать и анализировать выборку только с явным указанием 0, но в связи с ее малым размером данные могут быть сильно искажены.  
На мой взгляд предпочтительнее вариант 1, что реализовано далее.

In [50]:
df['Unregistered Vehicle?'].value_counts()

0.0    1127596
Name: Unregistered Vehicle?, dtype: int64

In [51]:
df['Unregistered Vehicle?'].isnull().sum()

9675432

In [52]:
df_corr['Unregistered Vehicle?'].fillna('0.0', inplace=True)

In [53]:
df_corr['Unregistered Vehicle?'].value_counts()

0.0    2782109
0.0        120
Name: Unregistered Vehicle?, dtype: int64

In [54]:
df_corr['Unregistered Vehicle?'].dtypes

dtype('O')

In [55]:
check_n = df_corr['House Number'].count()
print('осталось строк {:,d}'.format(check_n))
print('удалено строк {:,d}'.format(10803028 - check_n))

осталось строк 2,782,229
удалено строк 8,020,799
